# Infosys Quarterly Report Analysis comparison
In this demo, we explore answering complex queries by decomposing them into simpler sub-queries.

In [1]:
!pip install getpass4 pypdf llama-index openai faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00
  Created wheel for clipboard: filename=clipboard-0.0.4-py3-none-any.whl size=1847 sha256=7467c57cbe6013549983a9c56ba928903fbd53059960e49c5bfc8b4994aa29

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
import os
from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass("OpenAI Key:")

OpenAI Key:··········


In [5]:
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext
from llama_index.response.pprint_utils import pprint_response
from llama_index.llms import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [6]:
import faiss
from llama_index.vector_stores import FaissVectorStore

## Configure LLM service

In [7]:
llm = OpenAI(temperature=0,
             model="text-davinci-003",
             max_tokens=500)

service_context = ServiceContext.from_defaults(llm=llm)

### Multiple Documents

In [8]:
q1_2022_new = SimpleDirectoryReader(
    input_files=["/content/ifrs-inr-press-release_q1_2022.pdf"]
).load_data()

q1_2023_new = SimpleDirectoryReader(
    input_files=["/content/ifrs-inr-press-release_q1_2023.pdf"]
).load_data()

q1_2024_new = SimpleDirectoryReader(
    input_files=["/content/ifrs-inr-press-release_q1_2024.pdf"]
).load_data()

In [9]:
# dimensions of text-ada-embedding-002
d = 1536
faiss_index_m1 = faiss.IndexFlatL2(d)

vector_store_m1 = FaissVectorStore(faiss_index=faiss_index_m1)
storage_context_m1 = StorageContext.from_defaults(vector_store=vector_store_m1)
q1_2022_index_m = VectorStoreIndex.from_documents(q1_2022_new, storage_context=storage_context_m1)


In [10]:
# dimensions of text-ada-embedding-002
d = 1536
faiss_index_m2 = faiss.IndexFlatL2(d)

vector_store_m2 = FaissVectorStore(faiss_index=faiss_index_m2)
storage_context_m2 = StorageContext.from_defaults(vector_store=vector_store_m2)
q1_2023_index_m = VectorStoreIndex.from_documents(q1_2023_new, storage_context=storage_context_m2)


In [11]:
d = 1536
faiss_index_m3 = faiss.IndexFlatL2(d)

vector_store_m3 = FaissVectorStore(faiss_index=faiss_index_m3)
storage_context_m3 = StorageContext.from_defaults(vector_store=vector_store_m3)

q1_2024_index_m = VectorStoreIndex.from_documents(q1_2024_new, storage_context=storage_context_m3)

In [12]:
q1_2022__engine_m = q1_2022_index_m.as_query_engine(similarity_top_k=2)
q1_2023__engine_m = q1_2023_index_m.as_query_engine(similarity_top_k=2)
q1_2024__engine_m = q1_2024_index_m.as_query_engine(similarity_top_k=2)

In [13]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=q1_2022__engine_m,
        metadata=ToolMetadata(
            name="q1_2022",
            description="Provides quarterly performance report for infosys for first quarter of 2022",
        ),
    ),
    QueryEngineTool(
        query_engine=q1_2023__engine_m,
        metadata=ToolMetadata(
            name="q1_2023",
            description="Provides quarterly performance report for infosys for first quarter of 2023",
        ),
    ),
    QueryEngineTool(
        query_engine=q1_2024__engine_m,
        metadata=ToolMetadata(
            name="q1_2024",
            description="Provides quarterly performance report for infosys for first quarter of 2024",
        ),
    ),
]

In [14]:
s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [15]:
response = s_engine.query(
    "What was the revenue growth in the first quarters of 2022, 2023 and 2024?"
)

Generated 3 sub questions.
[q1_2022] Q: What was the revenue growth in the first quarter of 2022?
[q1_2023] Q: What was the revenue growth in the first quarter of 2023?
[q1_2024] Q: What was the revenue growth in the first quarter of 2024?
[q1_2024] A: The revenue growth in the first quarter of 2024 was not mentioned in the given context information.
[q1_2023] A: The revenue growth in the first quarter of 2023 was 21.4% year-on-year in constant currency terms.
[q1_2022] A: The revenue growth in the first quarter of 2022 was 16.9% year-on-year and 4.8% quarter-on-quarter in constant currency.


In [16]:
print(response)

The revenue growth in the first quarter of 2022 was 16.9% year-on-year and 4.8% quarter-on-quarter in constant currency. In the first quarter of 2023, the revenue growth was 21.4% year-on-year in constant currency terms. However, the revenue growth in the first quarter of 2024 was not mentioned in the given context information.


In [17]:
response = s_engine.query(
    "How the total assets have changed from 2022 to 2024 year by year?"
)

Generated 3 sub questions.
[q1_2022] Q: What are the total assets for Infosys in the first quarter of 2022?
[q1_2023] Q: What are the total assets for Infosys in the first quarter of 2023?
[q1_2024] Q: What are the total assets for Infosys in the first quarter of 2024?
[q1_2024] A: The total assets for Infosys in the first quarter of 2024 are 131,322 crore.
[q1_2022] A: The total assets for Infosys in the first quarter of 2022 are ₹109,486 crore.
[q1_2023] A: The total assets for Infosys in the first quarter of 2023 are ₹119,979 crore.


In [18]:
print(response)

The total assets for Infosys have increased from ₹109,486 crore in the first quarter of 2022 to ₹119,979 crore in the first quarter of 2023, and further increased to ₹131,322 crore in the first quarter of 2024.


In [19]:
response = s_engine.query(
    "Which are the key stragegic client wins or collborations that were achieved in 2022, 2023 and 2024?")

Generated 3 sub questions.
[q1_2022] Q: What were the key strategic client wins or collaborations achieved in 2022?
[q1_2023] Q: What were the key strategic client wins or collaborations achieved in 2023?
[q1_2024] Q: What were the key strategic client wins or collaborations achieved in 2024?
[q1_2022] A: The key strategic client wins or collaborations achieved in 2022 include Archrock, the French Tennis Federation, and Britvic. Archrock selected Infosys to integrate digital technologies and mobile tools for its field service technicians. The French Tennis Federation partnered with Infosys to transform the Roland-Garros experience for the tennis ecosystem. Britvic chose Infosys to help deliver their strategic transformation roadmap and operations.
[q1_2024] A: In 2024, Infosys achieved key strategic client wins and collaborations with companies such as Danske Bank, bp, Aramco, British Telecom, Walmart Commerce Technologies, vidaXL, and LexisNexis. These collaborations involved accelera

In [20]:
print(response)

The key strategic client wins or collaborations achieved in 2022 include Archrock, the French Tennis Federation, and Britvic. In 2023, Infosys achieved key strategic client wins and collaborations with Rolls-Royce, Backcountry, TK Elevator, and EisnerAmper. In 2024, Infosys achieved key strategic client wins and collaborations with Danske Bank, bp, Aramco, British Telecom, Walmart Commerce Technologies, vidaXL, and LexisNexis.


In [21]:
response = s_engine.query(
    "Which was the message by the CEO in the press release of first quarter for the year 2024?")

Generated 1 sub questions.
[q1_2024] Q: What is the message by the CEO in the press release of first quarter for the year 2024?
[q1_2024] A: The CEO's message in the press release for the first quarter of 2024 is that the company had a solid Q1 with a growth of 4.2% and large deals of $2.3 billion, which sets a strong foundation for future growth. The CEO also highlights the expansion of their generative AI capabilities, with 80 active client projects, and mentions that their comprehensive AI offering, Topaz, is resonating well with clients. The CEO sees this AI expansion as transformative for clients and enhancing the overall service portfolio. Additionally, the CEO mentions that the company has expanded the margin improvement program with a holistic set of actions for the short, medium, and long-term, supported by the leadership team.


In [22]:
print(response)

The CEO's message in the press release for the first quarter of 2024 is that the company had a solid Q1 with a growth of 4.2% and large deals of $2.3 billion, which sets a strong foundation for future growth. The CEO also highlights the expansion of their generative AI capabilities, with 80 active client projects, and mentions that their comprehensive AI offering, Topaz, is resonating well with clients. The CEO sees this AI expansion as transformative for clients and enhancing the overall service portfolio. Additionally, the CEO mentions that the company has expanded the margin improvement program with a holistic set of actions for the short, medium, and long-term, supported by the leadership team.
